In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import apogee.tools.read as apread
import pylab as pl
import os

class BFData:
      def __init__(self, locationID='', apogeeID='',ranger=''):
            folder = '/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder/'
            
            self.apogeeID = apogeeID
            self.locationID = locationID
            self.ranger = ranger
            self.filename = folder + str(ranger) + '/' + str(locationID) + '/' + str(apogeeID) + '.tbl'
            self.max1 = []
            self.max2 = []
            self.r = []

            data = np.loadtxt(self.filename, delimiter='\t', dtype=str,skiprows=1)
            for visit in data:
                self.max1.append(visit[2])
                self.max2.append(visit[4])
                self.r.append(visit[5:])


'''List = BFData(locationID,apogeeID,ranger)'''
List = BFData('4103', '2M08113137+3101105', 0.2)
print(List.apogeeID)
print(List.locationID)
print(List.ranger)
print(List.filename)

for visit in range(len(List.max1)):
        print('max1', visit + 1, List.max1[visit])

for visit in range(len(List.max2)):
        print('max2', visit + 1, List.max2[visit])
        
for visit in range(len(List.r)):
        print('r', visit + 1, List.r[visit])
'''for r in List.r:
        print(r)'''


folder = '/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder/'

def calcR(x, pos1=0, pos2=401):
        '''
        Calculates the value of R with the given array x
        :return:  The value of R
        '''        
        ccfCenter = 201
        pos1+= 1
        tempMirror = (x[ccfCenter:pos2])[::-1]
        sigmaA = np.sqrt(1.0 / (2.0 * len(tempMirror)) * np.sum((x[pos1:ccfCenter] - tempMirror)**2))
        return np.max(x) / (np.sqrt(2.0) * sigmaA)

def findInflection(x):
        '''
        Finds the inflection points of the given curve if available.
        :param x:  The curve to find the inflection points of.
        :return: The position of the inflection points (left, right)
        '''
        maxPos = np.argmax(x)
        pos1, pos2 = -1, -1
        for i in range(maxPos, len(x) - 1):
                if (x[i + 1] - x[i] > 0):
                        pos1 = i
                        break
        for i in range(maxPos, 1, -1):
                if (x[i - 1] - x[i] > 0):
                        pos2 = i
                        break
        return pos2, pos1

def getMaxPositions(x, yBufferRange):
        '''
        Gets the positions of both maximums (if two exists) in the given array
        :param x: The array to find both maximums
        :param yBufferRange:  The minimum difference needed between the two maximums
        :return:  The position of both maximums (max1, max2)
        '''
        temp = np.array(x)
        max1 = np.nanargmax(x)
        pos1, pos2 = findInflection(temp)
        temp[pos1:pos2] = np.nan
        temp[np.where(temp < 0.2)] = np.nan

        # If we can find one, record the second maximum
        try:
                max2 = np.nanargmax(temp)
                '''
                # Check to see wListch inflection point is closest to peak 2
                if (np.abs(x[max1] - x[max2]) < yBufferRange):
                        max2 = 'none'
                '''
                if (np.abs(max2 - pos1) < np.abs(max2 - pos2)):
                        # Check if it's witListn the yBufferRange
                        if (np.abs(x[max2] - x[pos1]) < yBufferRange):
                                max2 = 'none'
                elif (np.abs(max2 - pos1) > np.abs(max2 - pos2)):
                        if (np.abs(x[max2] - x[pos2]) < yBufferRange):
                                max2 = 'none'
        except ValueError:
                max2 = 'none'
        
        # Double check that we are returning different positions
        if str(max1) == str(max2):
                max2 = 'none'
        
        return max1, max2

def recordTargets(targets, ranger, filename):
        targetCount = len(targets)

        if not os.path.exists(folder + str(round(ranger, 2)) + '/'):
                os.makedirs(folder + str(round(ranger, 2)) + '/')
        filename = folder + str(round(ranger, 2)) + '/' + filename + '.csv'
        f = open(filename, 'w')

        for i in range(targetCount):
                f.write(str(targets[i][0]) + ',' + str(targets[i][1]) + '\n')
        f.close()

def reportPositions(locationID, apogeeID, ranger, positions):
        '''
        Records the positions of the maximums for each visit
        :param locationID:  The field ID of the target
        :param apogeeID: The apogee ID of the target
        :param ranger: The range used for yBufferRange in getMaxPositions
        :param positions: The positions of the (potentially two) maximums
        '''
        if positions == []:
                return
        
        visitCount = len(positions)

        if not os.path.exists(folder + str(round(ranger, 2)) + '/' + str(locationID) + '/'):
                os.makedirs(folder + str(round(ranger, 2)) + '/' + str(locationID) + '/')
        filename = folder + str(round(ranger, 2)) + '/' + str(locationID) + '/' + str(apogeeID) + '.tbl'
        f = open(filename, 'w')

        
        f.write('visit\tmax1\tmax2\t')
        for i in range(len(positions[0][2])):
                f.write('r'+str(i+1)+'\t\t\t')
        f.write('\n')

        for i in range(visitCount):
                f.write(str(i + 1))
                position = positions[i]
                # record postions of maximums
                f.write('\t\t' + str(position[0]))
                f.write('\t\t' + str(position[1]))
                # record r values
                for val in position[2]:
                        f.write('\t' + str(val))
                f.write('\n')
        f.close()
            
'''def plotCCFR(locationID, apogeeID, ranger, visit=1):
    hi = BFData(locationID, apogeeID, ranger)
    print(hi.apogeeID)
    print(hi.locationID)
    print(hi.ranger)
    print(hi.filename)

    for visit in range(len(hi.max1)):
            print('max1', visit + 1, hi.max1[visit])
    #Find the best R value representative of a given visit by searching for the largest separation in peaks.
    maxDiff = [0, 1]
    for visit in range(len(hi.max2)):
            print('max2', visit + 1, hi.max2[visit])
            if(hi.max2[visit] != "none"):
                  if(np.abs(int(hi.max2[visit]) - int(hi.max1[visit])) > maxDiff[0]):
                        maxDiff[0] = np.abs(int(hi.max2[visit]) - int(hi.max1[visit]))
                        maxDiff[1] = visit + 1                  
    print(maxDiff[0], maxDiff[1])

plotCCFR('4103', '2M08113137+3101105', 0.2)'''

In [ ]:
# Plotting the R Windows
import numpy as np
import matplotlib.pyplot as plt
import apogee.tools.read as apread
import pylab as pl
import os

class BFData:
      def __init__(self, locationID='', apogeeID='',ranger=''):
            folder = '/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder/'
            
            self.apogeeID = apogeeID
            self.locationID = locationID
            self.ranger = ranger
            self.filename = folder + str(ranger) + '/' + str(locationID) + '/' + str(apogeeID) + '.tbl'
            self.max1 = []
            self.max2 = []
            self.r = []

            data = np.loadtxt(self.filename, delimiter='\t', dtype=str,skiprows=1)
            for visit in data:
                self.max1.append(visit[2])
                self.max2.append(visit[4])
                self.r.append(visit[5:])
                
example = BFData('4103', '2M08113137+3101105', 0.2)
targetCount = 1
apogeeIDs=example.apogeeID
locationIDs=example.locationID
print(locationIDs)

plotname = 'Ranger 0.2_'+ str(locationIDs) +'_'+ str(apogeeIDs)
print(plotname)
TargetCount = len(locationIDs)

for i in range(TargetCount):
    badheader, header = apread.apStar(locationIDs,apogeeIDs, ext=0, dr='13', header=True)
    z= apread.apStar(locationIDs, apogeeIDs, ext=9, header=False, dr='13')
    nvisits = header['NVISITS']
    plt.figure(figsize=(20,20))
    
# Lists is for plotting the interestingTargets to visually confirm if the code is working appropriately.
    for visit in range(0, nvisits):
        if (nvisits != 1):
            CcF = z['CCF'][0][2+ visit]
        else:
            CcF = z['CCF'][0]
           
        #Conditions to adjust the legend size. For R_1 - R_5 and R_16-R_19 Window Calculations      
        if (visit < 6):
            fig = plt.figure(figsize=(20,20))
            sub1 = fig.add_subplot(331)
            sub1.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(10,391,color= 'green', alpha=0.05)
            plt.title('All Visits for '+ str(locationIDs)+'/'+str(apogeeIDs),fontsize=12)
            plt.ylabel('$\widehat{CCF}$ Units', fontsize=12)
            plt.text(220,1.5,'R_1 = ', fontsize=10)
            plt.legend(loc='lower left',fontsize=10)

            sub2 = fig.add_subplot(332)
            sub2.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(20,381,color='green', alpha=0.05)
            plt.text(220,1.5,'R_2 = ',fontsize=10)

            sub3 = fig.add_subplot(333)
            sub3.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(30,371,color='green', alpha=0.05)
            plt.text(220,1.5,'R_3 = ',fontsize=10)
            
            sub4 = fig.add_subplot(334)
            sub4.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(40,361,color='green', alpha=0.05)
            plt.text(220,1.5,'R_4 = ',fontsize=10)
            plt.ylabel('$\widehat{CCF}$ Units', fontsize=12)

            sub5 = fig.add_subplot(335)
            sub5.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(160,241,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{16} = ',fontsize=10)
        
            sub6 = fig.add_subplot(336)
            sub6.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(170,231,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{17} = ',fontsize=10)

            sub7 = fig.add_subplot(337)
            sub7.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(180,221,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{18} = ',fontsize=10)
            plt.xlabel('CCF Lag',fontsize=12)
            plt.ylabel('$\widehat{CCF}$ Units', fontsize=12)


            sub8 = fig.add_subplot(3,3,8)
            sub8.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(190,211,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{19} = ',fontsize=10)            
            plt.xlabel('CCF Lag',fontsize=12)

            sub9 = fig.add_subplot(3,3,9)
            sub9.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(200,201,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{20} = ',fontsize=10)
            plt.xlabel('CCF Lag',fontsize=12)
            plt.show()
            plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/M.02/'+str(plotname)+'.png')


                      
        else:
            fig = plt.figure(figsize=(20,20))
            sub1 = fig.add_subplot(331)
            sub1.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(10,391,color= 'green', alpha=0.05)
            plt.title('All Visits for '+ str(locationIDs)+'/'+str(apogeeIDs),fontsize=12)
            plt.ylabel('$\widehat{CCF}$ Units', fontsize=12)
            plt.text(220,1.5,'R_1 = ', fontsize=10)
            plt.legend(loc='upper right',fontsize=10)

            sub2 = fig.add_subplot(332)
            sub2.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(20,381,color='green', alpha=0.05)
            plt.text(220,1.5,'R_2 = ',fontsize=10)

            sub3 = fig.add_subplot(333)
            sub3.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(30,371,color='green', alpha=0.05)
            plt.text(220,1.5,'R_3 = ',fontsize=10)
            
            sub4 = fig.add_subplot(334)
            sub4.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(40,361,color='green', alpha=0.05)
            plt.text(220,1.5,'R_4 = ',fontsize=10)
            plt.ylabel('$\widehat{CCF}$ Units', fontsize=12)

            sub5 = fig.add_subplot(335)
            sub5.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(160,241,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{16} = ',fontsize=10)
        
            sub6 = fig.add_subplot(336)
            sub6.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(170,231,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{17} = ',fontsize=10)

            sub7 = fig.add_subplot(337)
            sub7.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(180,221,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{18} = ',fontsize=10)
            plt.xlabel('CCF Lag',fontsize=12)
            plt.ylabel('$\widehat{CCF}$ Units', fontsize=12)

            sub8 = fig.add_subplot(3,3,8)
            sub8.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            plt.axvspan(190,211,color='green', alpha=0.05)
            plt.text(220,1.5,'R_{19} = ',fontsize=10)            
            plt.xlabel('CCF Lag',fontsize=12)

            #sub9 = fig.add_subplot(3,3,9)
            #sub9.plot(CcF + visit,label= 'Visit: '+str(1+visit))
            #plt.axvspan(200,201,color='green', alpha=0.05)
           # plt.text(220,1.5,'R_{20} = ',fontsize=10)
            #plt.xlabel('CCF Lag',fontsize=12)
            plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/M.02/'+str(plotname)+'.png')
            

In [ ]:
   '''          plt.subplot(4,4,1)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(60,341,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(4,4,3)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(70,331,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(4,4,4)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(80,321,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(4,4,5)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(90,311,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(4,4,6)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(100,301,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)'''
               
                
             '''plt.subplot(5,6,1)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(110,291,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(5,6,3)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(120,281,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(5,6,4)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(130,271,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(5,6,5)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(140,261,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(5,6,6)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(150,251,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)'''
            
             '''plt.subplot(6,8,1)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(160,241,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(6,8,3)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(170,231,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(6,8,4)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(180,221,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(6,8,5)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(190,211,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
                
                plt.subplot(6,8,6)
                plt.plot(CcF + visit,label= 'Visit: '+str(1+visit))
                plt.axvspan(200,201,color='green', alpha=0.05)
                plt.xlabel('CCF Lag',fontsize=12)
                plt.text(220,1.5,'R = ',fontsize=10)
 
                #plt.savefig('/Volumes/CoveyData/APOGEE_Spectra/APOGEE2_DR13/Bisector/BinaryFinder_Plots/M.2/'+str(plotname)+'.png')
             '''